In [ ]:
## download Albert model 
# download data
sys.path.insert(0, '../')
from util import util_data_access
import zipfile
util_data_access.download_file('albert.zip', 'spark-nlp/albert_xxlarge_uncased_en_2.5.0_2.4_1588073588232.zip')
# ! tar -xf en_core_web_lg.tar.gz
# ! mv en_core_web_lg-2.3.1 en_core_web_lg
with zipfile.ZipFile('albert.zip', 'r') as zip_ref:
    zip_ref.extractall('albert')

# Load Spark

In [1]:
import sparknlp
from pyspark.sql import SparkSession
# spark = sparknlp.start()

spark = (
    SparkSession.builder \
      .config("spark.executor.instances", "50") \
      .config("spark.driver.memory", "15g") \
      .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.7.5")
      .getOrCreate()
)

In [2]:
spark

# Set up Spark-NLP

In [3]:
from pyspark.ml import PipelineModel, Pipeline
import sparknlp.annotator as sa
import sparknlp.base as sb
# import sparknlp
from sparknlp import Finisher

In [4]:
documenter = sb.DocumentAssembler()\
    .setInputCol("summary")\
    .setOutputCol("document")
    
# sentencerDL = sa.SentenceDetectorDLModel\
#     .load("/Users/alex/.cache/spark-nlp/sentence_detector_dl_en_2.6.2_2.4_1600002888450") \
#     .setInputCols(["document"]) \
#     .setOutputCol("sentences")

sentencer = (sa.SentenceDetector()
                .setInputCols(["document"])
                .setOutputCol("sentences")            
            )

finisher = Finisher() \
    .setInputCols(["sentences"]) \
#     .setIncludeMetadata(True)

sd_pipeline = PipelineModel(stages=[documenter, sentencer, finisher])

# Try Sentence Tokenizing on Our Own Data

In [5]:
import sqlite3
import pandas as pd
# import unidecode

In [23]:
# conn = sqlite3.connect('../data/diffengine-diffs/db/newssniffer-nytimes.db')
conn = sqlite3.connect('newssniffer-nytimes.db')

# df = pd.read_sql('''
#     SELECT * from entryversion 
#     WHERE entry_id IN (SELECT distinct entry_id FROM entryversion LIMIT 50000)
# ''', con=conn)

df = pd.read_sql('''
    SELECT * from entryversion 
''', con=conn)

In [29]:
chunksize = 40000
unique_entryids = df['entry_id'].unique()
num_chunks = int(unique_entryids.shape[0] / chunksize)

for chunk_id in range(num_chunks):
    

2

In [7]:
df = df.assign(summary=lambda df: df['summary'].str.replace('</p><p>', ' '))

In [8]:
sdf = spark.createDataFrame(df)

In [12]:
annotations_df = sd_pipeline.transform(sdf)

In [13]:
t_df = annotations_df.toPandas()

In [14]:
t_df['finished_sentences'][3]

['WASHINGTON — Weapons sales by the United States tripled in 2011 to a record high, driven by major arms sales to Persian Gulf allies concerned about Iran’s regional ambitions, according to a new study for Congress.',
 'Overseas weapons sales by the United States totaled $66.3 billion last year, or nearly 78 percent of the global arms market, valued at $85.3 billion in 2011.',
 'Russia was a distant second, with $4.8 billion in deals.',
 'The American weapons sales total was an “extraordinary increase” over the $21.4 billion in deals for 2010, the study found, and was the largest single-year sales total in the history of United States arms exports.',
 'The previous high was in fiscal year 2009, when American weapons sales overseas totaled nearly $31 billion.',
 'A worldwide economic decline had suppressed arms sales over recent years.',
 'But increasing tensions with Iran drove a set of Persian Gulf nations — Saudi Arabia, the United Arab Emirates and Oman — to purchase American weapon

# Get Albert Embeddings

In [ ]:
document_assembler = sb.DocumentAssembler()\
  .setInputCol("summary")\
  .setOutputCol("document")

tokenizer = sa.Tokenizer().setInputCols(["document"])\
  .setOutputCol("token")
 
word_embeddings = sa.AlbertEmbeddings.load('albert')\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")

In [19]:
ls /notebooks/edit-project/notebooks/albert/metadata/part-0000*

/notebooks/edit-project/notebooks/albert/metadata/part-00000


In [ ]:
word_embeddings = sa.AlbertEmbeddings.load('albert')\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")

In [ ]:
word_embeddings = sa.AlbertEmbeddings.load('albert')\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")

In [58]:
bert_pipeline = Pipeline().setStages(
  [
    document_assembler,
    tokenizer,
    word_embeddings
  ]
)

df_bert = bert_pipeline.fit(sdf).transform(sdf)

In [ ]:
t2_df = df_bert.toPandas()